<a href="https://colab.research.google.com/github/Gosshi/study_DNN/blob/Gosshi%2FCIFAR10_sample/CIFAR10/CIFAR10_practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import torch #基本モジュール
from torch.autograd import Variable #自動微分用
import torch.nn as nn #ネットワーク構築用
import torch.optim as optim #最適化関数
import torch.nn.functional as F #ネットワーク用の様々な関数
from torch.utils.data import DataLoader #データセット読み込み関連
import torchvision #画像関連
from torchvision import datasets, models, transforms #画像用データセット諸々
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

In [54]:
transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
CIFAR10_data = datasets.CIFAR10('~/tmp/CIFAR10', train = True, download = True, transform = transform)
data_loader = DataLoader(CIFAR10_data ,batch_size = 4, shuffle = False)
data_iterator = iter(data_loader)

images,labels = data_iterator.next()

Files already downloaded and verified


In [55]:
# 訓練データ
train_data_with_label = datasets.CIFAR10('~/tmp/CIFAR10',train = True, download = True, transform = transforms.ToTensor())
train_data_loader  = DataLoader(train_data_with_label, batch_size = 1000, shuffle = True)

# 検証データ
test_data_with_label = datasets.CIFAR10('~/tmp/CIFAR10',train = False, download = True, transform = transforms.ToTensor())
test_data_loader  = DataLoader(test_data_with_label, batch_size = 1000, shuffle = True)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [0]:
class mymodel(nn.Module):
 def __init__(self):
        super(mymodel, self).__init__()
        self.conv1 = nn.Conv2d(in_channels= 3, out_channels= 6, kernel_size= 5)
        self.pool = nn.MaxPool2d(2, 2)
        #self.drop = nn.Dropout(p = 0.2)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size= 5)
        #self.conv3 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size= 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 80)
        self.fc3 = nn.Linear(80,10)

 def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        # x = self.pool(F.relu(self.conv3(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [0]:
device = torch.device("cuda")

model = mymodel().cuda()

In [0]:
# 誤差関数と 最適化手法
loss_func = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = 0.01)

In [0]:
def train(epoch):
  model.train()
  
  for data, label in  train_data_loader:
    data, label = Variable(data).cuda(), Variable(label).cuda()
    optimizer.zero_grad()
    output = model(data)
    loss = loss_func(output,label)
    loss.backward()
    optimizer.step()
  print('epoch %d, loss %.4f' % (epoch, loss))

In [103]:
for epoch in range(100):
  train(epoch)

epoch 0, loss 0.9042
epoch 1, loss 0.9460
epoch 2, loss 1.0112
epoch 3, loss 0.9112
epoch 4, loss 1.0596
epoch 5, loss 0.9927
epoch 6, loss 0.9859
epoch 7, loss 0.9046
epoch 8, loss 0.8823
epoch 9, loss 0.9782
epoch 10, loss 0.9967
epoch 11, loss 0.9657
epoch 12, loss 0.9276
epoch 13, loss 0.9421
epoch 14, loss 0.8815
epoch 15, loss 0.9621
epoch 16, loss 0.9255
epoch 17, loss 0.9389
epoch 18, loss 0.9709
epoch 19, loss 0.9341
epoch 20, loss 0.9687
epoch 21, loss 0.9643
epoch 22, loss 0.9319
epoch 23, loss 0.9811
epoch 24, loss 0.9714
epoch 25, loss 0.9312
epoch 26, loss 0.9709
epoch 27, loss 0.9288
epoch 28, loss 0.9135
epoch 29, loss 1.0290
epoch 30, loss 0.9460
epoch 31, loss 0.9159
epoch 32, loss 0.9232
epoch 33, loss 0.9482
epoch 34, loss 0.9381
epoch 35, loss 0.9913
epoch 36, loss 0.9346
epoch 37, loss 0.9159
epoch 38, loss 0.8704
epoch 39, loss 0.9805
epoch 40, loss 0.9117
epoch 41, loss 0.8914
epoch 42, loss 0.8696
epoch 43, loss 0.9225
epoch 44, loss 0.9943
epoch 45, loss 0.963

In [0]:
def test():
  model.eval()
  correct = 0
  total = 0
  for data, label in test_data_loader:
    data, label = Variable(data).cuda(), Variable(label).cuda()
    outputs = model(data)
    _, predicted = torch.max(outputs.data, 1)
    total += label.size(0)
    correct += (predicted == label).sum().item()
 
  print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))
   

In [105]:
test()

Accuracy of the network on the 10000 test images: 48 %
